In [ ]:
pip install torch underthesea rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 21.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-ma

In [15]:
pip install rank_bm25

In [1]:
import json
import os
import re
import numpy as np
import torch


In [2]:
def load_all_data(input_file):
    with open(input_file, 'r', encoding='utf-8') as f:
      data = json.load(f)
    return data

# Mục mới

In [3]:

def clean_text(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

In [4]:
def preprocess_data(data):
    contexts = []
    contextRaws = []
    questions = []
    answers = []
    questionRaws = []
    for dataSquad in data:
      for article in dataSquad['data']:
          for paragraph in article['paragraphs']:
              context = clean_text(paragraph['context'])
              for qa in paragraph['qas']:
                  question = clean_text(qa['question'])
                  is_impossible = qa.get('is_impossible', False)  #Do ngữ liệu nhiều chỗ define thiếu is_impossible nên sẽ mặc định là False
                  if not is_impossible:
                      for answer in qa['answers']:
                          answer_text = clean_text(answer['text'])
                          answer_start = answer['answer_start']
                          contexts.append(context)
                          contextRaws.append(paragraph['context'])
                          questions.append(question)
                          questionRaws.append(qa['question'])
                          answers.append({
                              'text': answer_text,
                              'start': answer_start
                          })
    return contexts, questions, answers, contextRaws, questionRaws

In [5]:
def tokenize_texts(texts):
 return [text.split() for text in texts]

In [6]:
data_path = '/content/data/squad_2.0.json'
squad_data = load_all_data(data_path)
contexts, questions, answers, contextRaws, questionRaws = preprocess_data(squad_data)
tokenized_contexts = tokenize_texts(contexts)
tokenized_questions = tokenize_texts(questions)

In [ ]:
tokenized_questions[:100]

In [7]:
pip install gensim

In [8]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess


In [9]:
# Tạo mô hình Word2Vec
model_word2vec_context = Word2Vec(vector_size=100, window=10, min_count=1, sg=1, workers=4)
model_word2vec_question = Word2Vec(vector_size=100, window=10, min_count=1, sg=1, workers=4)

In [10]:
# Xây dựng từ điển
model_word2vec_context.build_vocab(tokenized_contexts)
# Xây dựng từ điển cho câu hỏi
model_word2vec_question.build_vocab(tokenized_questions)

In [11]:
model_word2vec_context.train(tokenized_contexts, total_examples=model_word2vec_context.corpus_count, epochs=200)

(2702269, 3854400)

In [12]:
model_word2vec_question.train(tokenized_questions, total_examples=model_word2vec_question.corpus_count, epochs=200)

(2750041, 4026400)

In [13]:
# Lưu mô hình Word2Vec đã huấn luyện
model_word2vec_context.save("/content/model_word2vec_context.model")
model_word2vec_question.save("/content/model_word2vec_question.model")

In [16]:
from rank_bm25 import BM25Okapi

# Tạo BM25 model
bm25 = BM25Okapi(tokenized_contexts)
bm25Questions = BM25Okapi(tokenized_questions)

In [17]:
# Changed
def get_avg_word2vec_vector(user_question, model_word2vec):
    words = user_question
    word_vectors = [model_word2vec.wv[word] for word in words if word in model_word2vec.wv]
    if not word_vectors:
        return np.zeros(model_word2vec.vector_size)
    return np.mean(word_vectors, axis=0)

In [18]:
# Changed

def get_word2vec_scores(user_question, datas, model_word2vec):
    query_vector = get_avg_word2vec_vector(user_question, model_word2vec)
    scores = []
    for data in datas:
        data_vector = get_avg_word2vec_vector(simple_preprocess(data), model_word2vec)
        score = np.dot(query_vector, data_vector)
        if np.isnan(score):
            score = 0
        scores.append(score)
    return np.array(scores)

In [19]:
# Added
def get_bm25_scores(user_question, bm25_model):
    bm_25_score = bm25_model.get_scores(user_question)
    scores = (bm_25_score - np.min(bm_25_score)) / (np.max(bm_25_score) - np.min(bm_25_score))
    # Thay thế giá trị nan bằng 0
    scores = np.nan_to_num(scores)
    return scores

In [20]:
# Added
def get_combined_scores(user_question, datas, model_word2vec, bm25_model):
    tokenized_query = user_question.split()
    word2vec_scores = get_word2vec_scores(tokenized_query, datas, model_word2vec)
    bm25_scores = get_bm25_scores(tokenized_query, bm25_model)
    word2vec_scores = (word2vec_scores - np.min(word2vec_scores)) / (np.max(word2vec_scores) - np.min(word2vec_scores))
    combined_scores = word2vec_scores + bm25_scores
    # Thay thế giá trị nan bằng 0 trong combined_scores
    combined_scores = np.nan_to_num(combined_scores)
    return combined_scores

In [21]:
def find_best_matching_question(user_question, questions, contextRaws, questionRaws):
    combined_scores = get_combined_scores(user_question, questions, model_word2vec_question, bm25Questions)
    if np.all(combined_scores == 0):
        return "Không có câu hỏi được tìm thấy"
    best_match_idx = np.argmax(combined_scores)
    return questionRaws[best_match_idx], contextRaws[best_match_idx], answers[best_match_idx]

user_question = "người sử dụng lao động cần làm gì về bảo hiểm xã hội"
closest_questions = find_best_matching_question(user_question, questions, contextRaws, questionRaws)
print(f"Closest Questions: {closest_questions}")

Closest Questions: ('Sinh viên tuyển sinh năm 2017 sử dụng quy chế tín chỉ nào số quyết định bao nhiêu?', 'Đối với khóa tuyển sinh từ năm 2016 trở về trước áp dụng theo Quy chế đào tạo theo hệ thống tín chỉ của Trường được ban hành theo Quyết định số 46/QĐ-DCT ngày 12/01/2016 của Hiệu trưởng Trường Đại học Công nghiệp Thực phẩm Tp. Hồ Chí Minh. Đối với khóa tuyển sinh từ năm 2017, 2018, 2019 áp dụng theo Quy chế đào tạo theo hệ thống tín chỉ của Trường được ban hành theo Quyết định số 1603/QĐ-DCT ngày 23/08/2017 của Hiệu trưởng Trường Đại học Công nghiệp Thực phẩm Tp. Hồ Chí Minh. Đối với khóa tuyển sinh năm 2020 áp dụng theo quy chế đào tạo theo hệ thống tín chỉ của Trường được ban hành theo Quyết định 2474/QĐ-DCT ngày 14/10/2020 của Hiệu trưởng Trường Đại học Công nghiệp Thực phẩm Tp. Hồ Chí Minh. Đối với khóa tuyển sinh từ năm 2021 trở đi áp dụng theo Quy chế đào tạo theo hệ thống tín chỉ của Trường được ban hành theo Quyết định số 1846/QĐ-DCT ngày 01/9/2021 của Hiệu trưởng Trường Đ

In [22]:
def find_best_matching_context(user_question, contexts):
    combined_scores = get_combined_scores(user_question, contexts, model_word2vec_context, bm25)
    if np.all(combined_scores == 0):
        return "Không có câu hỏi được tìm thấy"
    best_match_idx = np.argmax(combined_scores)
    return contexts[best_match_idx]

user_question = "Người sử dụng lao động phải làm gì về bảo hiểm xã hội"
closest_questions = find_best_matching_context(user_question, contexts)
print(f"Closest Questions: {closest_questions}")

Closest Questions: chuẩn đầu ra là yêu cầu tối thiểu về kiến thức kỹ năng thái độ trách nhiệm nghề nghiệp mà người học đạt được sau khi hoàn thành chương trình đào tạo được cơ sở đào tạo cam kết và người học xã hội và công bố công khai cùng với các điều kiện đảm bảo thực hiện chuẩn đầu ra của một chương trình đào tạo được tham chiếu theo khung trình độ quốc gia việt nam các quy định của bộ giáo dục và đào tạo và đảm bảo yếu tố hội nhập khu vực thế giới


In [23]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
path = "/content/drive/MyDrive/model_giaoduc"

In [26]:
# Tải mô hình phoBERT
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForQuestionAnswering.from_pretrained(path)


In [27]:
def get_answer(question):
    bestQuestion, contextMatch, answerMatch =find_best_matching_question(question, questions, contextRaws, questionRaws)
    inputs = tokenizer(bestQuestion.lower(), contextMatch, return_tensors="pt",max_length=128, padding="max_length", truncation="only_second")
    with torch.no_grad():
        outputs = model(**inputs)
    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()
    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    print("Context: " + contextMatch + "\n")
    print("Question " + bestQuestion+ "\n")
    answer_result = tokenizer.decode(predict_answer_tokens)

    if len(answerMatch['text'].strip()) > len(answer_result.strip()):
        print("Answer: " + answerMatch['text'] + "\n")
    elif(tokenizer.decode(predict_answer_tokens) == ""):
        print("Answer: Chưa thể tìm thấy câu trả lời \n")
    else:
        print("Answer: " + tokenizer.decode(predict_answer_tokens)+ "\n")


In [28]:
while(True):
    inputQuestion = input("Input context: \n")
    if (inputQuestion.lower() == "ok"):
        break
    get_answer(inputQuestion)

Input context: 
Trường Đại học Công nghiệp Thực phẩm Thành phố Hồ Chí Minh nay có tên là gì
Context: chào em; nếu em muốn chuyển ngành từ CN Thực phẩm sang CN Hóa; em làm đơn xin chuyển ngành theo mẫu http://sinhvien.hufi.edu.vn/NewsDetail.aspx?NewsID=19; nộp về Phòng đào tạo trước khi đăng ký môn học đk học ngành mới em nha

Question Em đổi ngành thực phẩm sang ngành hóa học á cô ơi

Answer: chào em nếu em muốn chuyển ngành từ cn thực phẩm sang cn hóa em làm đơn xin chuyển ngành theo mẫu httpsinhvienhufieduvnnewsdetailaspxnewsid19 nộp về phòng đào tạo trước khi đăng ký môn học đk học ngành mới em nha

Input context: 
Sau khi rút môn học thì sinh viên có phải đi học môn học đã rút không
Context: Người học chỉ được phép rút bớt học phần đã đăng ký: Thời gian rút học phần: trong 02 tuần (từ tuần thứ 03 đến tuần thứ 04 đối với học kỳ chính); trong 01 tuần (tuần thứ 02 đối với học kỳ hè). Môn học được rút sẽ được xóa thời khóa biểu, xóa tên trong danh sách lớp, không tính vào số tín chỉ đă